In [ ]:
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import PolynomialFeatures

class PolynomialFactorAnalysis:
    """
    Polynomial Factor Analysis is an extension of Factor Analysis that takes into consideration feature polynomial to capture non-linearities in the latent space, similarly to how Linear Regression Models use polinomials to fit the regressors on the regressand.
    """
    def __init__(self, degree=2, n_components=2, tol=0.001, max_iter=10000, 
                 noise_variance_init=None, svd_method='lapack', iterated_power=3, 
                 rotation='varimax', random_state=0, interaction_only=False, include_bias=True):
        """
        Initialize Polynomial Factor Analysis (PFA).
        Parameters:
        degree (int): The degree of the polynomial features transformation. 
                      It controls the highest degree of interaction for the polynomial features.
                      Default is 2 for quadratic features.

        n_components (int): The number of components (latent factors) to extract using Factor Analysis. 
                             This corresponds to the number of latent factors in the model. 
                             Default is 2.

        tol (float): The convergence tolerance for the Factor Analysis optimization algorithm. 
                     The algorithm will stop when the change in log-likelihood is less than this value. 
                     Default is 0.01.

        max_iter (int): The maximum number of iterations to perform in the Factor Analysis optimization.
                        Default is 1000.

        noise_variance_init (array-like, optional): The initial guess for the noise variance for each feature.
                                                    Default is None, which lets the model estimate it automatically.

        svd_method (str): The method to use for Singular Value Decomposition in Factor Analysis. 
                          Options are 'randomized' or 'lapack'. Default is 'randomized'.

        iterated_power (int): The number of iterations for the power method in the randomized SVD algorithm. 
                               Default is 3.

        rotation (str or None): The rotation method to apply to the latent factors. 
                                 Options include 'varimax' or 'quartimax'. Default is None, meaning no rotation.

        random_state (int or None): The seed for random number generation. Default is 0. 
                                    If None, the random seed will be chosen by the system.

        interaction_only (bool): If True, only interaction features (no polynomial terms like x^2, x^3) 
                                  will be included in the transformation. Default is False.

        include_bias (bool): If True, the transformed features will include a bias term (column of ones). 
                             Default is True.
        Attributes:
        poly (PolynomialFeatures): The polynomial feature transformer.
        fa (FactorAnalysis): The Factor Analysis model.
        """
        # Initialize PolynomialFeatures and FactorAnalysis
        self.degree = degree
        self.n_components = n_components
        self.tol = tol
        self.max_iter = max_iter
        self.noise_variance_init = noise_variance_init
        self.svd_method = svd_method
        self.iterated_power = iterated_power
        self.rotation = rotation
        self.random_state = random_state
        self.interaction_only = interaction_only
        self.include_bias = include_bias

        # Initialize PolynomialFeatures for polynomial expansion
        self.poly = PolynomialFeatures(degree=self.degree,
                                       interaction_only=self.interaction_only,
                                       include_bias=self.include_bias)

        # Initialize FactorAnalysis
        self.fa = FactorAnalysis(n_components=self.n_components,
                                 tol=self.tol,
                                 max_iter=self.max_iter,
                                 noise_variance_init=self.noise_variance_init,
                                 svd_method=self.svd_method,
                                 iterated_power=self.iterated_power,
                                 rotation=self.rotation,
                                 random_state=self.random_state)

    def fit(self, X):
        """
        Fit the Polynomial Factor Analysis model on the input data X.

        Parameters:
        X (array-like): The input data, where each row represents an observation and each column a feature.

        """
        # Apply polynomial feature transformation
        X_poly = self.poly.fit_transform(X)

        # Fit the Factor Analysis model to the polynomial transformed data
        self.fa.fit(X_poly)

    def transform(self, X):
        """
        Transform the data X using the fitted Polynomial Factor Analysis model.

        Parameters:
        X (array-like): The input data to transform.

        Returns:
        Transformed data (latent factors)
        """
        # Apply polynomial feature transformation
        X_poly = self.poly.transform(X)

        # Apply Factor Analysis to the polynomial transformed data
        return self.fa.transform(X_poly)

    def fit_transform(self, X):
        """
        Fit and transform the data using the Polynomial Factor Analysis model.

        Parameters:
        X (array-like): The input data.

        Returns:
        Transformed data (latent factors)
        """
        # Apply polynomial feature transformation
        X_poly = self.poly.fit_transform(X)

        # Apply Factor Analysis to the polynomial transformed data
        return self.fa.fit_transform(X_poly)

    def get_results(self, X):
        """
        Return AIC, BIC, and explained variance as a tuple.
        
        Parameters:
        X (array-like): The input data to calculate the results for.
        
        Returns:
        tuple: (AIC, BIC, explained_variance)
        """
        # Get the number of data points and features
        n_samples, n_features = X.shape

        # Log marginal likelihood
        log_marginal_likelihood = self.fa.log_marginal_likelihood_

        # Number of parameters: components (factor loadings) + noise variances
        n_parameters = self.n_components * n_features + n_features

        # Calculate AIC (Akaike's Information Criterion)
        AIC = 2 * n_parameters - 2 * log_marginal_likelihood
        
        # Calculate BIC (Bayesian Information Criterion)
        BIC = n_parameters * np.log(n_samples) - 2 * log_marginal_likelihood

        # Get the explained variance of the latent factors
        explained_variance = self.fa.explained_variance_

        # Return AIC, BIC, and explained variance as a tuple
        return (AIC, BIC, explained_variance, self.fa.components_ )

#### Searching for optima

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

# Sample data for fitting the model (replace with your actual dataset)
X = np.random.randn(100, 5)  # 100 samples, 5 features

# Initialize a list to store results
results_list = []

# Define the ranges for degrees and number of components
degree_range = [1, 2, 3]  # Example degrees
n_components_range = [1, 2, 3]  # Example components

# Iterate through all combinations of degree and n_components
for degree, n_components in product(degree_range, n_components_range):
    # Initialize and fit the PolynomialFactorAnalysis model
    pfa = PolynomialFactorAnalysis(degree=degree, n_components=n_components)
    pfa.fit(X)
    
    # Get results (AIC, BIC, and explained variance)
    AIC, BIC, explained_variance = pfa.get_results(X)
    
    # Store results as a tuple (degree, n_components, AIC, BIC, explained_variance)
    results_list.append((degree, n_components, AIC, BIC, explained_variance))

# Convert the results list into a DataFrame
results_df = pd.DataFrame(results_list, columns=["degree", "n_components", "AIC", "BIC", "explained_variance"])

# Show the resulting DataFrame
print(results_df)

#### Scree Plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the scree plot
plt.figure(figsize=(8, 6))
sns.lineplot(x='n_components', y='explained_variance', data=results_df, marker='o', color='blue')

# Add labels and title
plt.title("Scree Plot", fontsize=16)
plt.xlabel("Number of Components", fontsize=14)
plt.ylabel("Explained Variance", fontsize=14)
plt.xticks(df['n_components'])  # Ensure each component is labeled
plt.grid(True)

# Optionally: Mark the "elbow" if you want
# For example, you could highlight the "elbow" based on visual inspection or a rule (like variance > 0.1)
elbow_point = df[df['explained_variance'] > 0.1].iloc[-1]  # Example rule to mark elbow
plt.scatter(elbow_point['n_components'], elbow_point['explained_variance'], color='red', zorder=5, label="Elbow Point")
plt.legend()

# Show plot
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and test sets
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Initialize the PolynomialFactorAnalysis model (with desired degree and components)
pfa_model = PolynomialFactorAnalysis(degree=2, n_components=3)

# Fit the PFA model on the training data
pfa_model.fit(X_train)

# Make a copy of the test data (ground truth)
X_test_original = X_test.copy()

# Transform the test data using the fitted model
X_test_transformed = pfa_model.transform(X_test)

# Compare the transformed test data to the original test data
# Calculate Mean Squared Error (MSE) to assess how far the transformation has deviated
mse = mean_squared_error(X_test_original, X_test_transformed)

# Optionally, you can calculate other metrics like Euclidean distance, Cosine similarity, etc.
# For Euclidean distance, you can use numpy:
euclidean_distance = np.linalg.norm(X_test_original - X_test_transformed)

# If you want to use Cosine similarity, you can calculate that as well:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(X_test_original, X_test_transformed)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Euclidean Distance: {euclidean_distance}")
print(f"Cosine Similarity: {cosine_sim}")
